# PostgreSQL Practice

This notebook will contain the work associated with postgresqltutorial.com's python tutorial

Source: http://www.postgresqltutorial.com/postgresql-python/

## Step 1: Create and connect to the 'suppliers' database in terminal

Enter `/Applications/Postgres.app/Contents/Versions/12/bin/psql -p5432 "database_name"` where `database_name` is any existing database or the psql command if you already configured it

In [25]:
# THIS WAS THE ORIGINAL ATTEMPT IGNORE IT

# Steps adapted from https://stackoverflow.com/a/46877364

# Install psycopg2 if not already on your computer

# Step 1: Attempt to run the following in command line (Python 3):
# sudo apt install libpq-dev python3-dev

# A Java pop-up request may tell you:
# "To use the “apt” command-line tool you need to install a JDK." (Java Developer Kit)
# This is because this package is written in C and adapted for python

# You have a few options for each operating system compressed and uncompressed,
# I selected jdk-13.0.1_osx-x64_bin.dmg
# Run the sudo command after installation 



#!pip install psycopg2

# If you are still having trouble try updating setup tools
# !pip install --upgrade pip setuptools

#sudo -H pip3 install gitsome

In [26]:
# Adapted from https://stackoverflow.com/a/24645416

# You need homebrew for this version

# Step 1: Install Postgresql in terminal:
# brew install postgresql


# Step 2: Install psycopg2
# pip install psycopg2

#Step 3: Install config
# pip install config

In [27]:
import psycopg2
#from config import Config
from config import config

# Adapted from https://stackoverflow.com/a/58136849

In [28]:
# Connect in python to the database by creating a cursor object

# You can specify a "port" parameter, the default port is 5432

# conn = psycopg2.connect("dbname=suppliers user=postgres password=postgres")

# Or:

conn = psycopg2.connect(host="localhost",database="suppliers", user="postgres", password="postgres")

Instead of typing all parameters in the cursor you can use a `.ini` (initialization/configuration) file to store the information. This file will be written in a `postgresql` style format.

A function in the `config.py` file returns the parameters in a psycopg2 friendly format.

In [29]:
params = config()
print(params)

{'host': 'localhost', 'database': 'suppliers', 'user': 'postgres', 'password': 'postgres'}


In [30]:
def connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # read connection parameters
        params = config()
 
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)
      
        # create a cursor
        cur = conn.cursor()
        
   # execute a statement
        print('PostgreSQL database version:')
        cur.execute('SELECT version()')
 
        # display the PostgreSQL database server version
        db_version = cur.fetchone()
        print(db_version)
       
       # close the communication with the PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')

In [31]:
# https://stackoverflow.com/questions/419163/what-does-if-name-main-do

# This file is the main file, not the imported config file.

if __name__ == '__main__':
    connect()

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 12.0 on x86_64-apple-darwin16.7.0, compiled by Apple LLVM version 8.1.0 (clang-802.0.42), 64-bit',)
Database connection closed.


# Step 2: A Function That Executes Commands

In [35]:
# This function will execute commands NOT display results of queries!

def execute_commands(psql_commands):
    
    conn = None
    
    try:
        # read the connection parameters
        # The connection parameters are listed in the files
        # 'config.py' and 'database.ini'
        params = config()
        
        # connect to the PostgreSQL server
        # Info on args vs kwargs: http://book.pythontips.com/en/latest/args_and_kwargs.html
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        
        try:
            # If psql_commands is not a list
            cur.execute(psql_commands)
            
        except:
            # create table one by one
            for command in psql_commands:
                cur.execute(command)

        
        # close communication with the PostgreSQL database server
        cur.close()
        
        # commit the changes
        conn.commit()
        
    # Print error if any portion of the try statement fails
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    
    # Close the connection 
    finally:
        if conn is not None:
            conn.close()

# Step 3: Create Tables

Source: http://www.postgresqltutorial.com/postgresql-python/create-tables/

In [36]:
# This statement creates four tables within the 'supplies' database

""" create tables in the PostgreSQL database"""
create_table_commands = (
        """
        CREATE TABLE vendors (
            vendor_id SERIAL PRIMARY KEY,
            vendor_name VARCHAR(255) NOT NULL
        )
        """,
        """ CREATE TABLE parts (
                part_id SERIAL PRIMARY KEY,
                part_name VARCHAR(255) NOT NULL
                )
        """,
        """
        CREATE TABLE part_drawings (
                part_id INTEGER PRIMARY KEY,
                file_extension VARCHAR(5) NOT NULL,
                drawing_data BYTEA NOT NULL,
                FOREIGN KEY (part_id)
                REFERENCES parts (part_id)
                ON UPDATE CASCADE ON DELETE CASCADE
        )
        """,
        """
        CREATE TABLE vendor_parts (
                vendor_id INTEGER NOT NULL,
                part_id INTEGER NOT NULL,
                PRIMARY KEY (vendor_id , part_id),
                FOREIGN KEY (vendor_id)
                    REFERENCES vendors (vendor_id)
                    ON UPDATE CASCADE ON DELETE CASCADE,
                FOREIGN KEY (part_id)
                    REFERENCES parts (part_id)
                    ON UPDATE CASCADE ON DELETE CASCADE
        )
        """)

In [37]:
if __name__ == '__main__':
    execute_commands(create_table_commands)

In [41]:
params = config()
        
# connect to the PostgreSQL server
# Info on args vs kwargs: http://book.pythontips.com/en/latest/args_and_kwargs.html
conn = psycopg2.connect(**params)
cur = conn.cursor()

cur.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public';""")
print (cur.fetchall())

# for table in cur.fetchall():
#     print(table)
    
cur.close()
conn.commit()


[('parts',), ('part_drawings',), ('vendors',), ('vendor_parts',)]
